In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [7]:
df = pd.read_csv("/Users/dear/Data Science/Project/Data-Sci-project/results/data_formatted_subjectAreas.csv")
df["Processed Words"]

0        ['recent', 'developments', 'bifunctional', 'ai...
1        ['benefit', 'punishment', 'sensitivity', 'moto...
2        ['magic', 'biomarkers', 'predict', 'longterm',...
3        ['endocrine', 'disrupting', 'compounds', 'envi...
4        ['probabilistic', 'regular', 'grammar', 'infer...
                               ...                        
20211    ['prediction', 'diagnosis', 'chronic', 'kidney...
20212    ['incidence', 'healthcareassociated', 'urinary...
20213    ['turmericloaded', 'alginate', 'particulatebas...
20214    ['identification', 'novel', 'gut', 'microbiota...
20215    ['effectiveness', '1', '2per', 'cent', 'acetic...
Name: Processed Words, Length: 20216, dtype: object

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import ast

# Load the dataset
# df = pd.read_csv("/path/to/data_formatted_subjectAreas.csv")
df['Processed Words'] = df['Processed Words'].apply(ast.literal_eval)


# Prepare features and labels
X = df['Processed Words'].apply(lambda x: ' '.join(x))
y = df['Subject Areas']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF feature extraction
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Logistic Regression model
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = logreg.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.4443620178041543

Classification Report:
                                       precision    recall  f1-score   support

Agricultural and Biological Sciences       0.51      0.54      0.52       245
                 Arts and Humanities       0.50      0.05      0.10        37
                        Biochemistry       0.26      0.19      0.22       219
                            Business       0.47      0.21      0.29        67
                Chemical Engineering       0.42      0.23      0.30       119
                           Chemistry       0.29      0.33      0.31       241
                    Computer Science       0.46      0.54      0.50       257
                   Decision Sciences       0.00      0.00      0.00         4
                           Dentistry       0.61      0.26      0.37        53
        Earth and Planetary Sciences       0.50      0.12      0.19        60
            Econometrics and Finance       0.29      0.05      0.09        38
         

/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Load the dataset
# df = pd.read_csv("/path/to/data_formatted_subjectAreas.csv")
# df['Processed Words'] = df['Processed Words'].apply(eval)

# # Prepare features and labels
# X = df['Processed Words'].apply(lambda x: ' '.join(x))
# y = df['Subject Areas']

# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # TF-IDF feature extraction
# tfidf = TfidfVectorizer(max_features=5000)
# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [100, 500, 1000]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Train the model
grid_search.fit(X_train_tfidf, y_train)

# Best parameters and evaluation
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Evaluate the model
y_pred = best_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

Best Parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy: 0.4443620178041543

Classification Report:
                                       precision    recall  f1-score   support

Agricultural and Biological Sciences       0.51      0.54      0.52       245
                 Arts and Humanities       0.50      0.05      0.10        37
                        Biochemistry       0.26      0.19      0.22       219
                            Business       0.47      0.21      0.29        67
                Chemical Engineering       0.42      0.23      0.30       119
                           Chemistry       0.29      0.33      0.31       241
                    Computer Science       0.46      0.54      0.50       257
                   Decision Sciences       0.00      0.00      0.00         4
                           Dentistry       0.61      0.26      0.37        53
        Earth and Planetary Sciences       0.50      0.12      0.19        60
        

/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dear/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
